# Converting a Pytorch Model to ONNX

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms

import numpy as np
from PIL import Image
import time

In [ ]:
transform = transforms.Compose([transforms.Resize(224),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])]) 

device = torch.device(0 if torch.cuda.is_available() else 'cpu')

In [ ]:
# Load the pre-trained efficientnet_b1 model
res_net = models.efficientnet_b1(weights="IMAGENET1K_V2").to(device)
res_net.eval()

## How fast is inference?

In [ ]:
test_image = Image.open("../../data/dog.jpg")
test_image.resize((256, 256))

In [ ]:
# Convert to tensor
tensor_image = transform(test_image).unsqueeze(0)

In [ ]:
inference_time = []
with torch.no_grad():
    for _ in range(10):
        start_time = time.time()
        out_put = res_net(tensor_image.to(device))
        end_time = time.time()
        inference_time.append(end_time - start_time)
    
print("Minimum inference time %.4fs" % np.min(inference_time))

## Convert to ONNX

In [ ]:
 # Generate a random input tensor
test_input = torch.randn(1, 3, 224, 224, device=device)

# Export the model to ONNX format
torch.onnx.export(
    res_net,
    test_input,
    "efficientnet_b1.onnx",
    opset_version=12,
    export_params=True,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input' : {0 : 'batch_size'},
                  'output' : {0 : 'batch_size'}}
)